In [45]:
# import sys
# sys.path.append("..")
# from common_utils import DATA_HOME
import pandas as pd
import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from collections import Counter
# dataset = "playground-series-s4e2"


In [46]:
# DATA_HOME
train_data = pd.read_csv("/Users/anyxling/datasets/playground-series-s4e2/train.csv")
train_data.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [47]:
test_data = pd.read_csv('/Users/anyxling/datasets/playground-series-s4e2/test.csv')
test_data.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [48]:
# split x, y in the train dataset
X_train_data, y_train_data = train_data.iloc[:, :-1], train_data.iloc[:, -1]

# encode categorical variables for the features in train dataset 
str_cols_X_train = X_train_data.select_dtypes(include=['object']).columns
le_features = LabelEncoder()
for col in str_cols_X_train:
    X_train_data[col] = le_features.fit_transform(X_train_data[col])
X_train_data.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,1,3
1,1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,2,0
2,2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,2,3
3,3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,1,3
4,4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,1,3


In [49]:
# encode the labels in train data
le_labels = LabelEncoder()
y_train_data_encoded = le_labels.fit_transform(y_train_data)
y_train_data_encoded.shape, X_train_data.shape

((20758,), (20758, 17))

In [50]:
# encode categorical variables for the test data 
str_cols_test = test_data.select_dtypes(include=['object']).columns
for col in str_cols_test:
    test_data[col] = le_features.fit_transform(test_data[col])
test_data.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,2,0,2.825629,0,0.855400,0.000000,2,3
1,20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,2,0,3.000000,0,1.000000,0.000000,2,3
2,20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,2,0,2.621877,0,0.000000,0.250502,2,3
3,20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,2,0,2.786417,0,0.094851,0.000000,2,3
4,20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,2,0,2.653531,0,0.000000,0.741069,2,3


In [51]:
# scale the features and split it into train, validation, and test dataset
sc = StandardScaler()
X_train_data_scaled = sc.fit_transform(X_train_data)

X_train, X_val_test, y_train, y_val_test = train_test_split(X_train_data_scaled, y_train_data_encoded, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

((12454, 17), (4152, 17), (4152, 17))

In [52]:
# check if the dataset is imbalanced or not
train_class_distribution = Counter(y_train_data_encoded)
print(train_class_distribution)

Counter({4: 4046, 3: 3248, 1: 3082, 2: 2910, 0: 2523, 6: 2522, 5: 2427})


In [54]:
# use grid search to try out different combinations of parameters 
param_grid = {
    'C': [1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 1, 0.1]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro')
}

svm = SVC()
grid_search = GridSearchCV(svm, param_grid, scoring=scoring, refit='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1, 10, 100], 'gamma': ['scale', 'auto', 1, 0.1],
                         'kernel': ['linear', 'rbf']},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score, response_method='predict'),
                      'precision': make_scorer(precision_score, response_method='predict', average=macro),
                      'recall': make_scorer(recall_score, response_method='predict', average=macro)},
             verbose=1)

In [55]:
# get the best combination of parameters and its score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best score: 0.8649441926679018


In [56]:
# apply on the validation set 
best_model = grid_search.best_estimator_
val_accuracy = best_model.score(X_val, y_val)
print("Validation accuracy:", val_accuracy)

Validation accuracy: 0.8651252408477842


In [57]:
# compare the validation accuracy with the baseline model
dummy_clf = DummyClassifier(strategy='most_frequent') # naive classifier, majority vote
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_test)
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)
print("accuracy score for dummy classifier:", accuracy_dummy)

accuracy score for dummy classifier: 0.20086705202312138


In [58]:
# since the performance's good, apply on the test set
test_accuracy = best_model.score(X_test, y_test)
print("test accuracy:", test_accuracy)

test accuracy: 0.8682562620423893


In [59]:
test_data_scaled = sc.fit_transform(test_data)
test_pred = best_model.predict(test_data_scaled)
test_pred

array([3, 5, 4, ..., 0, 1, 3])

In [60]:
decoded_pred = le_labels.inverse_transform(test_pred)
decoded_pred

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype=object)

In [61]:
sub = pd.DataFrame({
    "id": test_data['id'],
    "NObeyesdad": decoded_pred
})
sub

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Overweight_Level_I
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [44]:
sub.to_csv('submission.csv', index=False)